In [1]:
import functools
import math
import pandas as pd
import random

### Use Pollard's Rho algorithm for finding prime factorizations, and thereby the list of all divisors, for any $n$

In [20]:
def fast_miller_rabin(n, use_probabilistic=False, tolerance=30):
    """
    Tests whether a number is prime using a deterministic version of the Miller-
    Rabin primality test. Optionally tests whether the specified number is a
    prime probabilistically up to a given tolerance using the regular version of
    the Miller-Rabin test. If the number is greater than 10^36, then all witnesses
    in the range [2, 2*log(n)*log(log(n))] are tested. However, this is conjectural
    and only heuristic evidence exists for it. To certify that a number is actually
    prime, one needs to test all witnesses in the range [2, 2*log(n)^2]. However,
    this is generally quite slow.
    Arguments:
        n (:int) - the integer to be tested
        use_probabilistic (:bool) - flag to indicate whether to use the regular
                                   version of the Miller-Rabin primality test
        tolerance (:int) - number of trials to be used to test primality
    Returns:
        True if 'n' is prime (or probably prime) and False otherwise
    References:
        - Francky from the PE Forums
        - https://miller-rabin.appspot.com/
        - https://en.wikipedia.org/wiki/Miller-Rabin_primality_test
    """
    firstPrime = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71]
    # Determine bases for deterministic Miller-Rabin test
    if n >= 10 ** 36:
        log_n = math.log(n)
        if not use_probabilistic:
            w = range(2, 2 * int(log_n * math.log(log_n) / math.log(2)))
        else:
            w = range(tolerance)
    elif n >= 1543267864443420616877677640751301:
        w = firstPrime[:20]
    elif n >= 564132928021909221014087501701:
        w = firstPrime[:18]
    elif n >= 59276361075595573263446330101:
        w = firstPrime[:16]
    elif n >= 6003094289670105800312596501:
        w = firstPrime[:15]
    elif n >= 3317044064679887385961981:
        w = firstPrime[:14]
    elif n >= 318665857834031151167461:
        w = firstPrime[:13]
    elif n >= 3825123056546413051:
        w = firstPrime[:12]
    # [2, 3, 5, 7, 11, 13, 17, 19, 23]
    elif n >= 341550071728321:
        w = firstPrime[:9]
    # [2, 3, 5, 7, 11, 13, 17]
    elif n >= 3474749660383:
        w = firstPrime[:7]
    elif n >= 2152302898749:
        w = firstPrime[:6]
    # [2, 3, 5, 7, 11, 13]
    elif n >= 4759123141:
        w = firstPrime[:5]
    # [2, 3, 5, 7, 11]
    elif n >= 9006403:
        w = [2, 7, 61]
    elif n >= 489997:
        # Some Fermat stuff
        if n & 1 and n % 3 and n % 5 and n % 7 and n % 11 and n % 13 and n % 17 and n % 19 and n % 23 and n % 29 and n % 31 and n % 37 and n % 41 and n % 43 and n % 47 and n % 53 and n % 59 and n % 61 and n % 67 and n % 71 and n % 73 and n % 79 and n % 83 and n % 89 and n % 97 and n % 101:
            hn, nm1 = n >> 1, n - 1
            p = pow(2, hn, n)
            if p == 1 or p == nm1:
                p = pow(3, hn, n)
                if p == 1 or p == nm1:
                    p = pow(5, hn, n)
                    return p == 1 or p == nm1
        return False
    elif n >= 42799:
        return n & 1 and n % 3 and n % 5 and n % 7 and n % 11 and n % 13 and n % 17 and n % 19 and n % 23 and n % 29 and n % 31 and n % 37 and n % 41 and n % 43 and pow(2, n - 1, n) == 1 and pow(5, n - 1, n) == 1
    elif n >= 841:
        return n & 1 and n % 3 and n % 5 and n % 7 and n % 11 and n % 13 and n % 17 and n % 19 and n % 23 and n % 29 and n % 31 and n % 37 and n % 41 and n % 43 and n % 47 and n % 53 and n % 59 and n % 61 and n % 67 and n % 71 and n % 73 and n % 79 and n % 83 and n % 89 and n % 97 and n % 101 and n % 103 and pow(2, n - 1, n) == 1
    elif n >= 25:
        return n & 1 and n % 3 and n % 5 and n % 7 and n % 11 and n % 13 and n % 17 and n % 19 and n % 23
    elif n >= 4:
        return n & 1 and n % 3
    else:
        return n > 1
    if not (n & 1 and n % 3 and n % 5 and n % 7 and n % 11 and n % 13 and n % 17
            and n % 19 and n % 23 and n % 29 and n % 31 and n % 37 and n % 41 and n % 43
            and n % 47 and n % 53 and n % 59 and n % 61 and n % 67 and n % 71 and n % 73
            and n % 79 and n % 83 and n % 89):
        return False
    # Miller-Rabin
    s = 0
    d = n - 1
    while not d & 1:
        d >>= 1
        s += 1
    for k in w:
        # Pick a random witness if probabilistic
        if use_probabilistic:
            p = random.randint(2, n - 2)
        else:
            p = k
        x = pow(p, d, n)
        if x == 1:
            continue
        for _ in range(s):
            if x + 1 == n:
                break
            x = x * x % n
        else:
            return False
    return True

# https://github.com/zhangbo2008/python_algorithm2/blob/c53669703b957a079f100c12711f86f5fc2f9389/algorithms/factorization/pollard_rho.py
def pollard_rho_prime_factorization(x):
    def f(x):
        return x * x + 1

    def rho(n, x1=2, x2=2):
        if n % 2 == 0:
            return 2
        i = 0
        while True:
            x1 = f(x1) % n
            x2 = f(f(x2)) % n
            divisor = math.gcd(abs(x1 - x2), n)
            i += 1
            if divisor != 1:
                break
            if i > 500:
                x1 = random.randint(1, 10)
                x2 = random.randint(1, 10)
                i = 0
        return divisor

    def pollard_rho_rec(x, factors):
        if x == 1:
            return

        if fast_miller_rabin(x):
            factors.append(x)
            return

        divisor = rho(x, random.randint(1, 10), random.randint(1, 10))
        pollard_rho_rec(divisor, factors)
        pollard_rho_rec(x // divisor, factors)

    if x == 1 or x == 0:
        return [(x, 1)]
    factors = []
    pollard_rho_rec(x, factors)
    return [(f, factors.count(f)) for f in set(factors)]

def divisor_list(n):
    # https://stackoverflow.com/questions/171765/what-is-the-best-way-to-get-all-the-divisors-of-a-number
    if n <= 1:
        return n
    
    factors = pollard_rho_prime_factorization(n)
    num_factors = len(factors)
    f = [0] * num_factors
    while True:
        yield functools.reduce(lambda x, y: x * y, [factors[x][0] ** f[x] for x in range(num_factors)], 1)
        i = 0
        while True:
            f[i] += 1
            if f[i] <= factors[i][1]:
                break
            f[i] = 0
            i += 1
            if i >= num_factors:
                return

### Form chains of numbers and their sums of proper divisors iteratively, remove and track separately completed chains from the search space in each iteration, and finally output the smallest element of the chain with the most elements

In [47]:
N = 10 ** 6
sum_proper_divisors = lambda n: sum(divisor_list(n)) - n
amicable_chain_df = pd.DataFrame({'n': list(range(1, N + 1)), 'sum_divisors': [sum_proper_divisors(n) for n in range(1, N + 1)]})
amicable_df = amicable_chain_df.copy()
amicable_chain_df.rename(columns={'n': 'n_0', 'sum_divisors': 'sum_divisors_0'}, inplace=True)
iteration, amicable_chains = 0, dict()
while len(amicable_chain_df) > 1:
    amicable_chain_df = pd.merge(amicable_chain_df, amicable_df.rename(columns={'n': f'n_{iteration + 1}', 'sum_divisors': f'sum_divisors_{iteration + 1}'}), left_on=f'sum_divisors_{iteration}', right_on=f'n_{iteration + 1}', how='inner')
    amicable_chains[iteration + 1] = (lambda x_df: (x_df.shape[0], [list(r)[1:] for r in x_df.itertuples()]))(amicable_chain_df.loc[amicable_chain_df.n_0 == amicable_chain_df[f'n_{iteration + 1}'], [col for col in amicable_chain_df.columns if col.startswith('n_')]])
    amicable_chain_df = amicable_chain_df[amicable_chain_df.apply(lambda r: r[f'n_{iteration + 1}'] not in [r[f'n_{i}'] for i in range(iteration + 1)], axis=1)]
    amicable_chain_df['chain_numbers'] = amicable_chain_df.apply(lambda r: '|'.join(sorted([str(r[f'n_{i}']) for i in range(iteration + 2)])), axis=1)
    amicable_chain_df.drop_duplicates(['chain_numbers'], inplace=True)
    amicable_chain_df.drop(['chain_numbers'], axis=1, inplace=True)
    iteration += 1
    print(iteration, len(amicable_chain_df))

for i in range(iteration, -1, -1):
    if amicable_chains[i][0] > 0:
        print('Longest Amicable chain:', amicable_chains[i][1][0])
        print('Smallest number in chain:', sorted(set(amicable_chains[i][1][0]))[0])
        break

1 937850
2 821214
3 715839
4 628651
5 553798
6 491950
7 439368
8 396947
9 362408
10 333412
11 307696
12 285778
13 267229
14 252312
15 239444
16 228180
17 217421
18 207380
19 197848
20 188732
21 179768
22 170682
23 161528
24 152268
25 142744
26 133056
27 123446
28 114157
29 105193
30 96544
31 88390
32 80569
33 73082
34 66166
35 59660
36 53699
37 48407
38 43760
39 39528
40 35821
41 32540
42 29615
43 27051
44 24755
45 22684
46 20842
47 19263
48 17832
49 16562
50 15329
51 14165
52 13056
53 11915
54 10816
55 9733
56 8780
57 7918
58 7096
59 6362
60 5717
61 5155
62 4631
63 4105
64 3604
65 3121
66 2687
67 2307
68 1939
69 1627
70 1337
71 1113
72 939
73 812
74 715
75 626
76 535
77 457
78 391
79 331
80 280
81 250
82 228
83 213
84 197
85 177
86 158
87 129
88 99
89 79
90 56
91 36
92 28
93 23
94 19
95 14
96 12
97 11
98 9
99 6
100 3
101 1
Longest Amicable chain: [19916, 17716, 14316, 19116, 31704, 47616, 83328, 177792, 295488, 629072, 589786, 294896, 358336, 418904, 366556, 274924, 275444, 243760, 37